## Load reservoir operation results conducted by Jia

This script loads Rdata of the hydropower operation results (Oct 2020) and save as
python preferred format.

In [1]:
import numpy as np
import pandas as pd
import os
os.environ['R_HOME'] = '/Users/dlee/anaconda3/envs/dlee/lib/R'  # The R path can be found by $ R RHOME
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects
pandas2ri.activate()

# Load Dam Inflow data from SUTD
dfFlowDams = pd.read_hdf('./data/dfFlowDams.hdf')
ind_dams = np.load('./data/ind_dams.npz')['ind_dams']
# Load Degree of Regulation (DOR)
dor = pd.read_hdf('./data/new_dor.hdf')      # The order is sorted during the process
dam_dor = dor.loc[dor.DOR2 <= 0, 'GRAND_ID']
# Select only headwater dams (735)
idx_dor = np.isin(ind_dams[0,:], dam_dor)
damList = ind_dams[0,idx_dor]
ind_dams = ind_dams[1,idx_dor]
ndam = len(damList)

In [15]:
# # Load dam operation results data
# rf = r['load']('./data/rdata/202010/simulation_2020_MP.Rdata')
# rf = r['simulation_2020_MP']
# # Total hydropower productiona
# prod = pd.DataFrame(np.zeros([ndam, 4]) , index=damList, columns=['dp', 'sdp', 'mpc_pf', 'mpc_df'])
# prod.index.name = 'GRAND_ID'
# for i in range(len(rf)):
#     did = np.array(rf[i][0])[0].astype(int)
#     prod.loc[did,'dp'] = np.round(rf[i][6][0][-1],5)     # DP
#     prod.loc[did,'sdp'] = np.round(rf[i][7][3][-1],5)    # SDP
#     prod.loc[did,'mpc_pf'] = np.round(rf[i][8][0][-1],5) # MPC_PF
#     prod.loc[did,'mpc_df'] = np.round(rf[i][9][0][-1],5) # MPC_DF
# # Calculate improvements
# prod['PF'] = (prod['mpc_pf'] - prod['sdp'])/prod['mpc_pf']*100
# prod['DF'] = (prod['mpc_df'] - prod['sdp'])/prod['mpc_pf']*100
# if True:
#     filn_prod = './data/hydropower_production.hdf'
#     prod.to_hdf(filn_prod, key='df', complib='blosc:zstd', complevel=9)
#     print('%s is saved.' % filn_prod)
    
# Import Jia's analysis results
reservoir = pd.read_csv('./data/rdata/202010/Analysis20201124/Reservoir.csv')
metrics = pd.read_csv('./data/rdata/202010/Analysis20201124/Res_metric.csv')
regression = pd.read_csv('./data/rdata/202010/Analysis20201124/Res_regression.csv')
prod = reservoir[['GRAND_ID', 'I_PF', 'I_DF']].set_index('GRAND_ID').rename({'I_PF':'PF', 'I_DF':'DF'}, axis=1)*100
filn_prod = './data/hydropower_production.hdf'
prod.to_hdf(filn_prod, key='df', complib='blosc:zstd', complevel=9)
print('%s is saved.' % filn_prod)

./data/hydropower_production.hdf is saved.


In [17]:
reservoir

,GRAND_ID,DAM_NAME,LONG_DD,LAT_DD,FLOOD,WEEK,DP_GWh,SDP_GWh,MPC_PF_GWh,MPC_DF_GWh,I_PF,I_DF,depth,fill,exceed,MdAPE,KGE
0,6,W.A.C. Bennett,-122.199583,56.020417,NaN,False,555343.646812,530526.792584,537231.071695,515140.013966,0.012479,-0.028641,1.000000,26.496765,0.093023,5.747356,0.763066
1,25,Strathcona,-125.587083,49.995417,NaN,False,15313.627271,14234.319410,15247.052481,13328.229931,0.066422,-0.059427,0.802407,5.051983,0.166667,24.243770,0.603888
2,27,Powell,-124.542083,49.887083,NaN,False,15608.685777,14885.011352,15392.260701,14934.452998,0.032955,0.003212,0.208756,8.603100,0.494186,23.888363,0.606360
3,31,Elsie Lake Saddle Dam,-125.102917,49.469583,NaN,True,6246.487863,5995.130007,6227.213096,5413.200006,0.037269,-0.093450,0.240277,1.682563,0.203488,22.861404,0.682417
4,35,Alouette,-122.482917,49.291250,NaN,False,2462.516460,2313.583811,2452.494684,2180.798686,0.056641,-0.054143,0.470880,4.373693,0.203488,19.564787,0.423303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,10022,Guangzhao Dam,105.250830,25.959440,NaN,False,126510.575644,115727.220171,125995.263772,103065.174854,0.081495,-0.100496,1.000000,5.384394,0.098837,24.887246,0.758769
731,10023,Jinping-I Dam,101.631514,28.182467,NaN,False,562137.999291,520075.054163,562081.758398,517024.229230,0.074734,-0.005428,1.000000,2.304382,0.129845,11.877959,0.894735
732,10024,Liyuan Dam,100.290000,27.661670,NaN,True,404499.405755,383549.109500,404501.085920,365683.430508,0.051797,-0.044167,1.000000,0.165833,0.211240,10.483494,0.943012
733,10025,Nam Theun 2 Dam,104.952780,17.996940,NaN,False,107557.840731,107073.584354,107295.645511,106663.537354,0.002070,-0.003822,0.137686,10.210677,0.060078,25.469846,0.646543
